<a href="https://colab.research.google.com/github/Angana1/Image-Segmentation/blob/main/Unet_Segmentation_New_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

# Model

In [ ]:
def conv_block(inputs, filters, pool=True):
    x = Conv2D(filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x
        
def build_unet(shape, num_classes):
    inputs = Input(shape)

    """ Encoder """
    x1, p1 = conv_block(inputs, 16, pool=True)
    x2, p2 = conv_block(p1, 32, pool=True)
    x3, p3 = conv_block(p2, 48, pool=True)
    x4, p4 = conv_block(p3, 64, pool=True)

    """ Bridge """
    b1 = conv_block(p4, 128, pool=False)

    """ Decoder """
    u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
    c1 = Concatenate()([u1, x4])
    x5 = conv_block(c1, 64, pool=False)

    u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
    c2 = Concatenate()([u2, x3])
    x6 = conv_block(c2, 48, pool=False)

    u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
    c3 = Concatenate()([u3, x2])
    x7 = conv_block(c3, 32, pool=False)

    u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
    c4 = Concatenate()([u4, x1])
    x8 = conv_block(c4, 16, pool=False)

    """ Output layer """
    output = Conv2D(num_classes, 1, padding="same", activation="softmax")(x8)

    return Model(inputs, output)

In [ ]:
model = build_unet((256, 256, 3), 3)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Preparing data

In [ ]:
train_x=[]
train_y=[]
valid_x=[]
valid_y=[]

#Train set : 55/60 images (i -> 0 to 54)
for i in range(55):
  s1='/content/img/sample_'+str(i+1)+'.JPG';
  train_x.append(s1)
  s2='/content/gt/sample_'+str(i+1)+'.JPG';
  train_y.append(s2)

#Test set : 5/60 images (i-> 56 to 60)
for i in range(55,60):
  s1='/content/img/sample_'+str(i+1)+'.JPG';
  valid_x.append(s1)
  s2='/content/gt/sample_'+str(i+1)+'.JPG';
  valid_y.append(s2)


'''train_x=['/content/img/sample_1.JPG','/content/img/sample_2.JPG','/content/img/sample_3.JPG','/content/img/sample_4.JPG','/content/img/sample_5.JPG',
         '/content/img/sample_6.JPG','/content/img/sample_7.JPG','/content/img/sample_8.JPG','/content/img/sample_9.JPG']
train_y=['/content/gt/sample_1.JPG','/content/gt/sample_2.JPG','/content/gt/sample_3.JPG','/content/gt/sample_4.JPG',
          '/content/gt/sample_5.JPG','/content/gt/sample_6.JPG','/content/gt/sample_7.JPG','/content/gt/sample_8.JPG','/content/gt/sample_9.JPG']
valid_x=['/content/img/sample_10.JPG']
valid_y=['/content/gt/sample_10.JPG']'''

"train_x=['/content/img/sample_1.JPG','/content/img/sample_2.JPG','/content/img/sample_3.JPG','/content/img/sample_4.JPG','/content/img/sample_5.JPG',\n         '/content/img/sample_6.JPG','/content/img/sample_7.JPG','/content/img/sample_8.JPG','/content/img/sample_9.JPG']\ntrain_y=['/content/gt/sample_1.JPG','/content/gt/sample_2.JPG','/content/gt/sample_3.JPG','/content/gt/sample_4.JPG',\n          '/content/gt/sample_5.JPG','/content/gt/sample_6.JPG','/content/gt/sample_7.JPG','/content/gt/sample_8.JPG','/content/gt/sample_9.JPG']\nvalid_x=['/content/img/sample_10.JPG']\nvalid_y=['/content/gt/sample_10.JPG']"

In [ ]:
print(valid_x, len(valid_x))

['/content/img/sample_56.JPG', '/content/img/sample_57.JPG', '/content/img/sample_58.JPG', '/content/img/sample_59.JPG', '/content/img/sample_60.JPG'] 5


In [ ]:
H = 256
W = 256

In [ ]:
def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.int32)
    return x

def tf_dataset(x, y, batch=1):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    #dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    #dataset = dataset.repeat()
    #dataset = dataset.prefetch(2)
    return dataset

def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        image = read_image(x)
        mask = read_mask(y)

        return image, mask

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
    #mask = tf.one_hot(mask, 3, dtype=tf.int32)
    image.set_shape([H, W, 3])
    mask.set_shape([H, W, 3])

    return image, mask

In [ ]:
dataset = tf_dataset(train_x, train_y, batch=1)
print(len(dataset))
for x, y in dataset:
  print(x.shape, y.shape) 

55
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 256, 3) (1, 256, 256, 3)
(1, 256, 25

In [ ]:
print(len(dataset))
#dataset

55


# Training

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
shape = (256, 256, 3)
num_classes = 3
lr = 1e-2
batch_size = 1
epochs = 100


In [ ]:
 """ Model """
model = build_unet(shape, num_classes)
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(lr))

In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
train_steps = len(train_x)//batch_size   #number of batches
valid_steps = len(valid_x)//batch_size   #number of batches

In [ ]:
callbacks = [
        ModelCheckpoint("/content/model.h5", verbose=1, save_best_model=True),
        ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-4)
     
    ]
# EarlyStopping(monitor="val_loss", patience=5, verbose=1)

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_data=valid_dataset,
        validation_steps=valid_steps,
        epochs=epochs,
        callbacks=callbacks
    )

Epoch 1/100
55/55 [==============================] - 35s 58ms/step - loss: 0.2910 - val_loss: 287.0493

Epoch 00001: saving model to /content/model.h5
Epoch 2/100
55/55 [==============================] - 3s 48ms/step - loss: 0.2846 - val_loss: 102.6596

Epoch 00002: saving model to /content/model.h5
Epoch 3/100
55/55 [==============================] - 3s 48ms/step - loss: 0.2851 - val_loss: 5.3432

Epoch 00003: saving model to /content/model.h5
Epoch 4/100
55/55 [==============================] - 3s 49ms/step - loss: 0.2836 - val_loss: 0.3305

Epoch 00004: saving model to /content/model.h5
Epoch 5/100
55/55 [==============================] - 3s 50ms/step - loss: 0.2826 - val_loss: 4.0717

Epoch 00005: saving model to /content/model.h5
Epoch 6/100
55/55 [==============================] - 3s 49ms/step - loss: 0.2791 - val_loss: 0.5815

Epoch 00006: saving model to /content/model.h5
Epoch 7/100
55/55 [==============================] - 3s 48ms/step - loss: 0.2813 - val_loss: 0.3028

Epoch 

# Testing

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import cv2

In [ ]:
for x, y in tqdm(zip(valid_x, valid_y), total=len(valid_x)):
        name = x.split("/")[-1]

        ## Read image
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        x = cv2.resize(x, (W, H))
        x = x / 255.0
        x = x.astype(np.float32)
        print('shape of x',x.shape)

        ## Read mask
        y = cv2.imread(y, cv2.IMREAD_COLOR)
        y = cv2.resize(y, (W, H))
        y = y / 255.0
        y = y.astype(np.float32)
        print('shape of y',y.shape)
        '''y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        y = cv2.resize(y, (W, H))   ## (256, 256)
        y = y - 1
        y = np.expand_dims(y, axis=-1) ## (256, 256, 1)
        y = y * (255/num_classes)
        y = y.astype(np.int32)
        y = np.concatenate([y, y, y], axis=2)'''

        ## Prediction
        p = model.predict(np.expand_dims(x, axis=0))
        print('shape 1:',p.shape )
        p = np.argmax(p, axis=-1)
        print('shape 2:',p.shape )
        p = np.expand_dims(p, axis=-1)
        print('shape 3:',p.shape )
        p = p * (255/num_classes)
        print('shape 4:',p.shape )
        p = p.astype(np.int32)
        #p = np.concatenate([p, p, p], axis=2)



        x = x * 255.0
        x = x.astype(np.int32)

        h, w, _ = x.shape
        line = np.ones((h, 10, 3)) * 255

        # print(x.shape, line.shape, y.shape, line.shape, p.shape)

        #final_image = np.concatenate([x, line, y, line, p], axis=1)
        final_image=p[0]
        print('shape of p',p.shape)
        print('shape of final image',final_image.shape)
        cv2.imwrite(f"/content/results/{name}", final_image)

  0%|          | 0/5 [00:00<?, ?it/s]

shape of x (256, 256, 3)
shape of y (256, 256, 3)


 60%|██████    | 3/5 [00:00<00:00,  2.63it/s]

shape 1: (1, 256, 256, 3)
shape 2: (1, 256, 256)
shape 3: (1, 256, 256, 1)
shape 4: (1, 256, 256, 1)
shape of p (1, 256, 256, 1)
shape of final image (256, 256, 1)
shape of x (256, 256, 3)
shape of y (256, 256, 3)
shape 1: (1, 256, 256, 3)
shape 2: (1, 256, 256)
shape 3: (1, 256, 256, 1)
shape 4: (1, 256, 256, 1)
shape of p (1, 256, 256, 1)
shape of final image (256, 256, 1)
shape of x (256, 256, 3)
shape of y (256, 256, 3)
shape 1: (1, 256, 256, 3)
shape 2: (1, 256, 256)
shape 3: (1, 256, 256, 1)
shape 4: (1, 256, 256, 1)
shape of p (1, 256, 256, 1)
shape of final image (256, 256, 1)
shape of x (256, 256, 3)
shape of y (256, 256, 3)


100%|██████████| 5/5 [00:00<00:00,  6.16it/s]

shape 1: (1, 256, 256, 3)
shape 2: (1, 256, 256)
shape 3: (1, 256, 256, 1)
shape 4: (1, 256, 256, 1)
shape of p (1, 256, 256, 1)
shape of final image (256, 256, 1)
shape of x (256, 256, 3)
shape of y (256, 256, 3)
shape 1: (1, 256, 256, 3)
shape 2: (1, 256, 256)
shape 3: (1, 256, 256, 1)
shape 4: (1, 256, 256, 1)
shape of p (1, 256, 256, 1)
shape of final image (256, 256, 1)


In [ ]:
#final_image = np.concatenate([line, p], axis=1)
#cv2.imwrite("/content/results.JPG", final_image)